# korQuAD 데이터셋 파인튜닝

## 모델 다운로드

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

- unsloth: 경량화된 LLM 로드 및 훈련을 지원하는 라이브러리
- max_seq_length: 모델이 한 번에 처리할 수 있는 최대 입력 길이
- load_in_4bit = True: 4비트 양자화를 적용하여 메모리 사용량을 줄이고 속도를 개선

In [ ]:
# Base Model 세팅

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length, # 최대 토큰 길이 설정 (모델이 한 번에 처리할 수 있는 최대 입력 길이)
    dtype = dtype,
    load_in_4bit = load_in_4bit, # 4-bit 양자화(Quantization) 적용 여부
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.5.1+cu124)
    Python  3.11.9 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

**LoRA(Low-Rank Adaptation) 적용**

- r: LoRA 랭크 설정 (랭크가 낮으면 학습 파라미터가 줄어들어 더 가볍지만, 너무 낮으면 성능이 저하될 수 있음)

- lora_alpha: LoRA 스케일링 계수 (LoRA 가중치를 스케일링하는 파라미터, 크면 클수록 LoRA 가중치 업데이트가 강해짐)

- lora_dropout: LoRA 층에서 사용할 드롭아웃 비율

- bias: Bias 학습 여부 ("none"이면 모델의 기존 Bias 파라미터를 업데이트하지 않음)

In [ ]:
# lora adapter 사용

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## 데이터 전처리
**KorQuAD 데이터셋을 LLM 입력 형식(prompt template) 으로 변환**
- examples["question"]: 질문 (ex. "대한민국의 수도는 어디인가?")
- examples["context"]: 본문(보기) (ex. "대한민국의 수도는 서울특별시이다.")
- examples["answers"]: 정답 리스트 (ex. {"text": ["서울특별시"]})
- 모델이 텍스트 생성을 마쳤음을 인식하도록 EOS(End of Sentence) 토큰을 추가


In [ ]:
# KorQuAD 데이터셋으로 파인튜닝

KorQuAD_prompt = """

### 질문:
{}

### 보기:
{}

### 답변:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs = examples["context"]
    outputs = [item['text'][0] for item in examples["answers"]]

    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = KorQuAD_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    # print(texts[0])

    return {"text": texts}
pass

In [ ]:
from datasets import load_dataset

# KorQuAD 데이터셋 로드
train_dataset = load_dataset("KorQuAD/squad_kor_v1", split="train")
val_dataset = load_dataset("KorQuAD/squad_kor_v1", split="validation")

# print(train_dataset[0])

# Formatting 적용
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

# 데이터 개수 확인
print(f"Train 데이터 개수: {len(train_dataset)}")
print(f"Validation 데이터 개수: {len(val_dataset)}")

README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Map:   0%|          | 0/60407 [00:00<?, ? examples/s]

Map:   0%|          | 0/5774 [00:00<?, ? examples/s]

Train 데이터 개수: 60407
Validation 데이터 개수: 5774


In [ ]:
train_dataset[0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]},
 'text': '\n\n### 질문:\n바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n\n### 보기:\n1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다

## 모델 학습

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/60407 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 60,407 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 300
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jubin-402 (jubin-402-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


## 모델 평가

In [ ]:
text_streamer = TextStreamer(tokenizer)

while True:
    print("질문을 입력하세요.")
    question = input()

    if question == "":
        print("QnA를 종료합니다.")
        break

    print("보기를 입력하세요.")
    context = input()

    inputs = tokenizer(
        [
            KorQuAD_prompt.format(
                question,  # instruction
                context,   # input
                "",        # output
            )
        ],
        return_tensors="pt",
        padding="longest",
        truncation=True
    ).to("cuda")

    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    print()

## 분석 및 고찰

### Meta-Llama-3.2-1B

- SFTTrainer() (LoRA 사용 용이)를 사용하려고 하니, 데이터 format을 SFTTrainer()에 맞게 다시 수정해주어야 했습니다.
  - Trainer:
    - "input_ids": prompt_tokenized["input_ids"] (prompt = 질문 + 지문 + 정답 + end token)
    - "attention_mask": prompt_tokenized를 모두 1로 마스킹한 것
    - "labels": labels (prompt 중, 질문 + 지문 + 정답만 -100으로 masking 한 것, 모델이 정답만 학습하도록 설정)
    - LoRA를 적용하려면 PEFT 라이브러리를 직접 활용하여, LoRA 모델을 수동으로 설정한 후 Trainer에 넣어야 함 (peft_get_model)

  - SFTTrainer:
    - "text": "질문 + 지문 + 정답 + end token" (내부적으로 이를 input_ids와 labels로 변환하여 정답 부분을 예측하도록 학습)
    - QLoRA와 PEFT를 지원하는 구조로 되어 있어, 별도의 설정 없이도 LoRA 적용이 가능
    - SFTTrainer는 text 입력만으로 자동으로 정답 부분을 학습하여, 별도의 masking과 label 작업이 필요없음


- 데이터 전처리를 하는 과정이 현재 작성한 코드와 너무 달라, LoRA를 사용하지 않으려고 했습니다.

- 학습시간이 너무 오래 걸려 다시 LoRA를 사용하기로 하였습니다.

- 양자화된 모델에 LoRA를 사용하려고 해보았으나, dlpc 환경에서 버전을 맞추기 어려웠습니다.

- 과제를 마치고 생각해보니, 양자화된 모델에 LoRA를 적용하는 것은 dlpc에서 지원을 해주지 않는 것 같습니다. (QLoRA를 사용해야 함)

### Meta-Llama-3.1-8B

- 4-bit 양자화 모델을 적용하여 메모리를 절감하고 연산량을 감소하였습니다.

- LoRA Adapter 적용하여(r=16, lora_alpha=16) 모델 전체가 아닌 핵심 레이어(q_proj, k_proj 등)만 미세 조정하였습니다.

- KorQuAD 데이터셋을 자연어 프롬프트 형식으로 변환해보았습니다. Meta-Llama-3.2-1B에서는 제대로 수행하지 못한 전처리를 "text" 필드로 변환하여 SFTTrainer에서 사용할 수 있도록 설정하였습니다.

- 적은 step으로 모델을 학습하였을 때는, 언더피팅이 발생하였습니다. step 많이 늘렸을 때는, 모델이 커서 금방 오버피팅이 발생하는 현상이 나타났습니다.

- 크기가 작은 모델을 사용하거나, 적절한 하이퍼파라미터 튜닝값을 필요로 할 것으로 예상됩니다.

## 추가로 알게 된 것
- QLoRA는 양자회된 모델을 메모리 효율적으로 파인튜닝하기 위한 방법으로 나온 것입니다.
- 양자화 되지 않은 모델에 LoRA를 적용할 수는 있으나, 메모리 사용량이 크고, VRAM이 제한적인 환경에서는 부담이 됩니다.
- 따라서, VRAM을 절약하면서도 LoRA를 적용하고 싶다면, 모델을 4-bit로 양자화한 후 QLoRA를 사용하는 것이 효과적입니다.
- LoRA와 QLoRA의 학습되는 파라미터 수는 동일하나, QLoRA가 메모리 사용량이 더 적습니다.
- QLoRA는 메모리를 절약하는 대신, 연산 비용이 증가하는 트레이드오프를 가집니다.
- 양자화된 모델을 효과적으로 파인튜닝하기 위해 QLoRA가 등장했지만, 초기 시도에서는 단순히 양자화된 모델에 LoRA를 적용하려 했고, 이는 DLPC에서 지원되지 않았던 것으로 보입니다.